In [1]:
import csv
import numpy as np

with open('/Users/JacobHsieh/Desktop/wireshark_analysis/max_evcc_UDP_TCP.csv', newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary
    rows = csv.DictReader(csvfile)
    lengthlist = []
    timelist = []
    seqlist = []
    tcpseglenlist = []
    
    for row in rows:
        lengthlist.append(int(row['Length']))
        timelist.append(float(row['Time']))
        seqlist.append(row['Sequence Number'])
        tcpseglenlist.append(row['TCP Segment Len'])

In [2]:
round_split_length = []
round_split_time = []
round_split_seq = []
round_split_tcpseglen = []

round_start_point = 0
round_end_point = 0

for i in range(1, len(lengthlist)):
    if(lengthlist[i-1] == 72 and lengthlist[i] == 90):
        round_end_point = i-1
        if(round_end_point != 0):
            tmp = []
            tmp2 = []
            tmp3 = []
            tmp4 = []
            for j in range(round_start_point, round_end_point):
                tmp.append(lengthlist[j])
                tmp2.append(timelist[j])
                tmp3.append(seqlist[j])
                tmp4.append(tcpseglenlist[j])
            round_split_length.append(tmp)
            round_split_time.append(tmp2)
            round_split_seq.append(tmp3)
            round_split_tcpseglen.append(tmp4)
            
        round_start_point = i-1
        
    if(i == len(lengthlist) - 1):
        round_end_point = i
        tmp = []
        tmp2 = []
        tmp3 = []
        tmp4 = []
        for j in range(round_start_point, round_end_point):
            tmp.append(lengthlist[j])
            tmp2.append(timelist[j])
            tmp3.append(seqlist[j])
            tmp4.append(tcpseglenlist[j])
        round_split_length.append(tmp)
        round_split_time.append(tmp2)
        round_split_seq.append(tmp3)
        round_split_tcpseglen.append(tmp4)

In [3]:
#for debug
print(len(round_split_length))
print(len(round_split_time))
print(len(round_split_seq))
print(len(round_split_tcpseglen))

20
20
20
20


In [ ]:
#for debug
for i in range(0, len(round_split_length)):
    print(len(round_split_length[i]))
    print(len(round_split_time[i]))
    print(len(round_split_seq[i]))
    print(len(round_split_tcpseglen[i]))

In [ ]:
#for debug
round_split_tcpseglen[0]

In [4]:
#preprocessing
for i in range(0, len(round_split_seq)):
    for j in range(0, len(round_split_seq[i])):
        if(round_split_seq[i][j] == ''):
            round_split_seq[i][j] = -1
        else: round_split_seq[i][j] = int(round_split_seq[i][j])
        if(round_split_tcpseglen[i][j] == ''):
            round_split_tcpseglen[i][j] = -1
        else: round_split_tcpseglen[i][j] = int(round_split_tcpseglen[i][j]) 

In [23]:
#for debug
round_split_seq[9]

[-1,
 -1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 91,
 203,
 1519,
 203,
 1529,
 203,
 203,
 1658,
 245,
 1658,
 251,
 1658,
 1658,
 336,
 1664,
 336,
 336,
 1749,
 1749,
 437,
 437,
 1818,
 1818,
 522,
 522,
 1919,
 1919,
 607,
 607,
 2084,
 2084,
 692,
 692,
 2120,
 789,
 2169,
 2169,
 2169,
 2217,
 2217,
 2270,
 2270,
 2590,
 2590,
 2355,
 2355,
 2691,
 2691,
 2691,
 2760,
 2760,
 2776,
 2776,
 2845,
 2845,
 2845,
 2861,
 2861,
 2962,
 2962,
 2946,
 2946,
 3079,
 3079,
 3031,
 3031,
 3196,
 3196,
 3116,
 3116,
 3313,
 3313,
 3201,
 3201,
 3430,
 3430,
 3286,
 3286,
 3547,
 3547,
 3371,
 3371,
 3664,
 3664,
 3456,
 3456,
 3781,
 3781,
 3541,
 3541,
 3898,
 3898,
 3626,
 3626,
 4015,
 4015,
 3711,
 3711,
 4132,
 4132,
 3796,
 3796,
 4217,
 4217,
 3881,
 3881,
 3950,
 4302,
 4302,
 4302,
 4371,
 3951,
 3951,
 4372]

In [44]:
time_anchor = []
stage_num = 0 #for debug
for i in range(0, len(round_split_time)): #Test Round i
    print("--------- Test Round " + str(i+1) + " ---------")
    time_anchor_each_round = []
    #Catching the Re-transmission
    is_req_found_flag = False
    is_res_found_flag = False
    for j in range(0, len(round_split_time[i])):        
        if(j == 0):
            #start, Stage1(SDP) Start
            time_anchor_each_round.append(round_split_time[i][j])
            if(stage_num != 0):
                print("Error.")
                break
            else:
                print("Stage 0 finish.")
                stage_num = stage_num + 1
            
        if(round_split_seq[i][j] == -1 and round_split_seq[i][j+1] == 0):
            #Stage1 (SDP) finish
            #And Stage 2 Start (TCP SYN)
            time_anchor_each_round.append(round_split_time[i][j])
            time_anchor_each_round.append(round_split_time[i][j+1])
            if(stage_num != 1):
                print("Error.")
                break
            else:
                print("Stage 1 finish. " + str(len(time_anchor_each_round)))
                stage_num = stage_num + 1
        
        if(round_split_seq[i][j] >= 1660 and round_split_seq[i][j] <= 1670):
            #Stage2 (Session Initialized) finish
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 2):
                    print("Error.")
                    break
                else:
                    print("Stage 2 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 1660~1670, len != 85")
                    
        if(round_split_seq[i][j] >= 330 and round_split_seq[i][j] <= 340):
            #Stage3 (SupportedAppProtocol) Start
            if(round_split_tcpseglen[i][j] == 101):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 3):
                    print("Error.")
                    break
                    
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 330~340, len != 101")
        
        if(round_split_seq[i][j] >= 1745 and round_split_seq[i][j] <= 1755):
            #Stage3 (SupportedAppProtocol) finish
            if(round_split_tcpseglen[i][j] == 69):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 3):
                    print("Error.")
                    break
                else:
                    print("Stage 3 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 1745~1755, len != 69")
        
        if(round_split_seq[i][j] >= 435 and round_split_seq[i][j] <= 440):
            #Stage4 (SessionSetup) Start
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 4):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 435~440, len != 85")
        
        if(round_split_seq[i][j] >= 1815 and round_split_seq[i][j] <= 1825):
            #Stage4 (SessionSetup) finish
            if(round_split_tcpseglen[i][j] == 101):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 4):
                    print("Error.")
                    break
                else:
                    print("Stage 4 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 1815~1825, len != 101")
        
        if(round_split_seq[i][j] >= 520 and round_split_seq[i][j] <= 555):
            #Stage5 (ServiceDiscovery) Start
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 5):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 520~555, len != 85")
        
        if(round_split_seq[i][j] >= 1915 and round_split_seq[i][j] <= 1925):
            #Stage5 (ServiceDiscovery) finish
            if(round_split_tcpseglen[i][j] == 165):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 5):
                    print("Error.")
                    break    
                else:
                    print("Stage 5 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 1915~1925, len != 165")
        
        if(round_split_seq[i][j] >= 605 and round_split_seq[i][j] <= 610):
            #Stage6 (ServicePaymentSelection) Start
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 6):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 605~610, len != 85")
        
        if(round_split_seq[i][j] >= 2075 and round_split_seq[i][j] <= 2085):
            #Stage6 (ServicePaymentSelection) finish
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 6):
                    print("Error.")
                    break
                else:
                    print("Stage 6 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2075~2085, len != 85")
        if(round_split_seq[i][j] >= 2115 and round_split_seq[i][j] <= 2125):
            #Stage7 (ServicePaymentSelection) Start
            if(round_split_tcpseglen[i][j] == 97):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 7):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2115~2125, len != 97")
        
        if(round_split_seq[i][j] >= 2165 and round_split_seq[i][j] <= 2175):
            #Stage7 (PaymentDetail) finish
            if(round_split_tcpseglen[i][j] == 101):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 7):
                    print("Error.")
                    break
                else:
                    print("Stage 7 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2165~2175, len != 101")
        
        if(round_split_seq[i][j] >= 2215 and round_split_seq[i][j] <= 2220):
            #Stage8 (Authorization) Start
            if(round_split_tcpseglen[i][j] == 373):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 8):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2215~2220, len != 373")
        
        if(round_split_seq[i][j] >= 2265 and round_split_seq[i][j] <= 2275):
            #Stage8 (Authorization) finish
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 8):
                    print("Error.")
                    break
                else:
                    print("Stage 8 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2265~2275, len != 85")
        
        if(round_split_seq[i][j] >= 2585 and round_split_seq[i][j] <= 2595):
            #Stage9 (ChargeParameterDiscovery) Start
            if(round_split_tcpseglen[i][j] == 101):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 9):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2585~2595, len != 101")
        
        if(round_split_seq[i][j] >= 2350 and round_split_seq[i][j] <= 2360):
            #Stage9 (ChargeParameterDiscovery) finish
            if(round_split_tcpseglen[i][j] == 405):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 9):
                    print("Error.")
                    break
                else:
                    print("Stage 9 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2350~2360, len != 405")
        
        if(round_split_seq[i][j] == 2691):
            #Stage10 (PowerDelivery) Start
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                if(stage_num != 10):
                    print("Error.")
                    break
                
                if(is_req_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = True
                is_res_found_flag = False
                
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2690~2695, len != 85")
        
        if(round_split_seq[i][j] >= 2755 and round_split_seq[i][j] <= 2765):
            #Stage10 (PowerDelivery) finish
            if(round_split_tcpseglen[i][j] == 85):
                time_anchor_each_round.append(round_split_time[i][j])
                
                if(is_res_found_flag):
                    print("Retransmission? " + str(round_split_time[i][j]))
                is_req_found_flag = False
                is_res_found_flag = True
                
                if(stage_num != 10):
                    print("Error.")
                    break
                else:
                    print("Stage 10 finish. " + str(len(time_anchor_each_round)))
                    stage_num = stage_num + 1
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + round_split_tcpseglen[i][j])
                    print("Seq# 2755~2765, len != 85")
    
    stage_num = 0
    time_anchor.append(time_anchor_each_round)

--------- Test Round 1 ---------
Stage 0 finish.
Stage 1 finish. 3
Stage 2 finish. 4
Stage 3 finish. 6
Stage 4 finish. 8
Stage 5 finish. 10
Stage 6 finish. 12
Stage 7 finish. 14
Stage 8 finish. 16
Stage 9 finish. 18
Stage 10 finish. 20
--------- Test Round 2 ---------
Stage 0 finish.
Stage 1 finish. 3
Stage 2 finish. 4
Stage 3 finish. 6
Stage 4 finish. 8
Stage 5 finish. 10
Stage 6 finish. 12
Stage 7 finish. 14
Stage 8 finish. 16
Stage 9 finish. 18
Stage 10 finish. 20
--------- Test Round 3 ---------
Stage 0 finish.
Stage 1 finish. 3
Stage 2 finish. 4
Stage 3 finish. 6
Stage 4 finish. 8
Stage 5 finish. 10
Stage 6 finish. 12
Stage 7 finish. 14
Stage 8 finish. 16
Stage 9 finish. 18
Stage 10 finish. 20
--------- Test Round 4 ---------
Stage 0 finish.
Stage 1 finish. 3
Stage 2 finish. 4
Stage 3 finish. 6
Stage 4 finish. 8
Stage 5 finish. 10
Stage 6 finish. 12
Stage 7 finish. 14
Stage 8 finish. 16
Stage 9 finish. 18
Stage 10 finish. 20
--------- Test Round 5 ---------
Stage 0 finish.
Stage 1

In [45]:
#for debug
print(len(time_anchor))
for i in range(0, len(time_anchor)):
    print(len(time_anchor[i]))

20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20


In [46]:
#Retransmission packet will be caught
len(time_anchor[9])

21

In [47]:
time_anchor[9]

[288.624889,
 288.716709,
 289.421518,
 289.799358,
 289.851004,
 289.869902,
 290.00715,
 290.024838,
 290.032043,
 290.052421,
 290.263896,
 290.330017,
 290.625863,
 290.801795,
 290.98708,
 291.046644,
 291.056688,
 291.08152,
 291.33247,
 291.466733,
 291.468921]

In [48]:
#Retransmission Dealing
time_anchor[9].remove(291.466733)

In [49]:
len(time_anchor[9])

20

In [52]:
time_anchor[0]

[13.93106,
 14.055571,
 14.836977,
 15.11966,
 15.193262,
 15.311269,
 15.472655,
 15.496974,
 15.512115,
 15.535569,
 16.202245,
 16.331248,
 16.535155,
 16.701734,
 16.814188,
 16.857827,
 16.871466,
 16.923864,
 17.013524,
 17.071535]

In [54]:
path = 'pcap_processing.txt'
f = open(path, 'w')
for i in range(0, len(time_anchor)):
    for j in range(1, len(time_anchor[i]), 2):
        print((time_anchor[i][j] - time_anchor[i][j-1])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((time_anchor[i][3] - time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((time_anchor[i][len(time_anchor[i])-1] - time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()